# MNIST 
## 1. as a image

In [81]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn,functional
import torch.nn.functional as F

In [82]:
np.random.seed(1) # seed
df_train = pd.read_csv(r"./mnist_as_csv/train.csv") 
df_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
X = df_train.loc[:,df_train.columns !="label"]
Y = df_train.loc[:,df_train.columns =="label"]

In [84]:
X = X/255
X =X.to_numpy()

In [85]:
Y = Y.to_numpy()

In [86]:
x_train, x_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=0)

In [87]:
BATCH_SIZE =16

X_train = torch.from_numpy(x_train)
X_test = torch.from_numpy(x_test)

Y_train = torch.from_numpy(y_train).type(torch.LongTensor) 
Y_test = torch.from_numpy(y_test).type(torch.LongTensor)

train = torch.utils.data.TensorDataset(X_train,Y_train)
test = torch.utils.data.TensorDataset(X_test,Y_test)

train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = True)

# model

## 1.1DNN

In [185]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN,self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64) 
        self.fc3 = nn.Linear(64, 10)
        self.relu= nn.ReLU()
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        
        return F.log_softmax(x, dim=1)

In [218]:
class DNN_drop(nn.Module):
    def __init__(self):
        super(DNN_drop,self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.dropout= nn.Dropout(0.1)
        self.fc2 = nn.Linear(128, 64) 
        self.fc3 = nn.Linear(64, 10)
        self.relu= nn.ReLU()
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        
        
        return F.log_softmax(x, dim=1)

In [219]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 2)
        self.relu = nn.ReLU()
        
        self.fc1 = nn.Linear(4374, 128) 
        self.fc2 = nn.Linear(128, 64) 
        self.out = nn.Linear(64, 10) 
        
    def forward(self,x):
        
        out = self.conv(x)
        out = self.relu(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.out(out)
        
        return F.log_softmax(out, dim=1)

In [220]:
from torchsummary import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_1 = DNN().to(device)
model_2 = DNN_drop().to(device)
model_3 = CNN().to(device)
# model_1=model_1.double()
# model_2=model_2.double()
# model_3=model_3.double()


In [221]:
summary(model_1, (1, 28*28))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 128]              100,480
├─ReLU: 1-2                              [-1, 1, 128]              --
├─Linear: 1-3                            [-1, 1, 64]               8,256
├─ReLU: 1-4                              [-1, 1, 64]               --
├─Linear: 1-5                            [-1, 1, 10]               650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (M): 0.11
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.42
Estimated Total Size (MB): 0.42
------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 128]              100,480
├─ReLU: 1-2                              [-1, 1, 128]              --
├─Linear: 1-3                            [-1, 1, 64]               8,256
├─ReLU: 1-4                              [-1, 1, 64]               --
├─Linear: 1-5                            [-1, 1, 10]               650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (M): 0.11
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.42
Estimated Total Size (MB): 0.42
------------------------------------------------------------------------------------------

In [222]:
summary(model_2, (1, 28*28))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 128]              100,480
├─ReLU: 1-2                              [-1, 1, 128]              --
├─Dropout: 1-3                           [-1, 1, 128]              --
├─Linear: 1-4                            [-1, 1, 64]               8,256
├─ReLU: 1-5                              [-1, 1, 64]               --
├─Linear: 1-6                            [-1, 1, 10]               650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (M): 0.11
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.42
Estimated Total Size (MB): 0.42
------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Linear: 1-1                            [-1, 1, 128]              100,480
├─ReLU: 1-2                              [-1, 1, 128]              --
├─Dropout: 1-3                           [-1, 1, 128]              --
├─Linear: 1-4                            [-1, 1, 64]               8,256
├─ReLU: 1-5                              [-1, 1, 64]               --
├─Linear: 1-6                            [-1, 1, 10]               650
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
Total mult-adds (M): 0.11
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.42
Estimated Total Size (MB): 0.42
------------------------------------------------------------------------------------------

In [223]:
summary(model_3, (1, 28, 28))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 6, 27, 27]           30
├─ReLU: 1-2                              [-1, 6, 27, 27]           --
├─Linear: 1-3                            [-1, 128]                 560,000
├─Linear: 1-4                            [-1, 64]                  8,256
├─Linear: 1-5                            [-1, 10]                  650
Total params: 568,936
Trainable params: 568,936
Non-trainable params: 0
Total mult-adds (M): 0.59
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 2.17
Estimated Total Size (MB): 2.21
------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 6, 27, 27]           30
├─ReLU: 1-2                              [-1, 6, 27, 27]           --
├─Linear: 1-3                            [-1, 128]                 560,000
├─Linear: 1-4                            [-1, 64]                  8,256
├─Linear: 1-5                            [-1, 10]                  650
Total params: 568,936
Trainable params: 568,936
Non-trainable params: 0
Total mult-adds (M): 0.59
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.03
Params size (MB): 2.17
Estimated Total Size (MB): 2.21
------------------------------------------------------------------------------------------

In [224]:
# criterion =   nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_1.parameters(),lr=0.001)

# dataset

### custom dataset과 image folder를 활용해보자

In [96]:
# class MNIST_CSV(Dataset):
#     def __init__(self, csv_file, transform=None):

#         self.mnist = pd.read_csv(csv_file)
#         self.transform = transform

#     def __len__(self):
#         return len(self.mnist)

#     def __getitem__(self, idx):
#         sample = self.mnist[idx]

#         if self.transform:
#             sample = self.transform(sample)

#         return sample

In [98]:
# from torchvision.datasets import ImageFolder 
# # #dataset 먼저 선언 후 loader에 맵핑
# train_dataset = ImageFolder(root='./mnist_as_image', transform=None) 
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)



NameError: name 'ImageFolder' is not defined

## without dropout

In [225]:
total_step = len(train_loader)
for epoch in range(15):
    model_1.train()
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device).float()
        labels = labels.to(device)
        labels = labels.squeeze(1)
        
        optimizer.zero_grad()
        outputs = model_1(images)
        loss = F.nll_loss(outputs, labels)
        
        
        loss.backward()
        optimizer.step()
        

            
    with torch.no_grad():
        correct = 0
        total = 0
        model_1.eval()
        for images, labels in test_loader:
            images = images.to(device).float()
            labels = labels.to(device)
            labels = labels.squeeze(1)
            outputs = model_1(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 94.85714285714286 %
Accuracy of the network on the 10000 test images: 96.41666666666667 %
Accuracy of the network on the 10000 test images: 96.38095238095238 %
Accuracy of the network on the 10000 test images: 97.20238095238095 %
Accuracy of the network on the 10000 test images: 96.61904761904762 %
Accuracy of the network on the 10000 test images: 96.97619047619048 %
Accuracy of the network on the 10000 test images: 97.32142857142857 %
Accuracy of the network on the 10000 test images: 97.05952380952381 %
Accuracy of the network on the 10000 test images: 97.36904761904762 %
Accuracy of the network on the 10000 test images: 97.26190476190476 %
Accuracy of the network on the 10000 test images: 96.94047619047619 %
Accuracy of the network on the 10000 test images: 97.38095238095238 %
Accuracy of the network on the 10000 test images: 97.27380952380952 %
Accuracy of the network on the 10000 test images: 96.80952380952381 %
Accuracy of the netw

In [226]:
optimizer = torch.optim.Adam(model_2.parameters(),lr=0.001)

## with dropout

In [227]:
total_step = len(train_loader)
for epoch in range(15):
    model_2.train()
    for i, (images, labels) in enumerate(train_loader):  
    
        images = images.to(device).float()
        labels = labels.to(device)
        labels = labels.squeeze(1)
        
        optimizer.zero_grad()
        outputs = model_2(images)
        loss = F.nll_loss(outputs, labels)
#         print("loss is :%f" % loss.item())
        loss.backward()
        optimizer.step()
        

            
    with torch.no_grad():
        correct = 0
        total = 0
        model_2.eval()
        for images, labels in test_loader:
            images = images.to(device).float()
            labels = labels.to(device)
            labels = labels.squeeze(1)
            outputs = model_2(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 95.0952380952381 %
Accuracy of the network on the 10000 test images: 96.08333333333333 %
Accuracy of the network on the 10000 test images: 96.57142857142857 %
Accuracy of the network on the 10000 test images: 96.76190476190476 %
Accuracy of the network on the 10000 test images: 96.85714285714286 %
Accuracy of the network on the 10000 test images: 96.66666666666667 %
Accuracy of the network on the 10000 test images: 97.08333333333333 %
Accuracy of the network on the 10000 test images: 97.32142857142857 %
Accuracy of the network on the 10000 test images: 97.28571428571429 %
Accuracy of the network on the 10000 test images: 97.08333333333333 %
Accuracy of the network on the 10000 test images: 97.58333333333333 %
Accuracy of the network on the 10000 test images: 96.9047619047619 %
Accuracy of the network on the 10000 test images: 97.58333333333333 %
Accuracy of the network on the 10000 test images: 97.4047619047619 %
Accuracy of the network

In [228]:
criterion =   nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_3.parameters(),lr=0.001)

In [229]:
total_step = len(train_loader)
for epoch in range(15):
    model_3.train()
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1,1,28,28).to(device).float()
        labels = labels.to(device)
        labels = labels.squeeze(1)
        
        optimizer.zero_grad()
        outputs = model_3(images)
        loss = F.nll_loss(outputs, labels)
        
        loss.backward()
        optimizer.step()
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, 15, i+1, total_step, loss.item()))

        

            
    with torch.no_grad():
        correct = 0
        total = 0
        model_3.eval()
        for images, labels in test_loader:
            images = images.reshape(-1,1,28,28).to(device).float()
            labels = labels.to(device)
            labels = labels.squeeze(1)
            outputs = model_3(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Epoch [1/15], Step [100/2100], Loss: 0.3657
Epoch [1/15], Step [200/2100], Loss: 0.4869
Epoch [1/15], Step [300/2100], Loss: 0.0685
Epoch [1/15], Step [400/2100], Loss: 0.3513
Epoch [1/15], Step [500/2100], Loss: 0.4647
Epoch [1/15], Step [600/2100], Loss: 0.6733
Epoch [1/15], Step [700/2100], Loss: 0.4573
Epoch [1/15], Step [800/2100], Loss: 0.3677
Epoch [1/15], Step [900/2100], Loss: 0.4958
Epoch [1/15], Step [1000/2100], Loss: 0.2925
Epoch [1/15], Step [1100/2100], Loss: 0.3718
Epoch [1/15], Step [1200/2100], Loss: 0.5895
Epoch [1/15], Step [1300/2100], Loss: 0.0935
Epoch [1/15], Step [1400/2100], Loss: 0.6530
Epoch [1/15], Step [1500/2100], Loss: 0.1517
Epoch [1/15], Step [1600/2100], Loss: 0.4574
Epoch [1/15], Step [1700/2100], Loss: 0.5392
Epoch [1/15], Step [1800/2100], Loss: 0.0719
Epoch [1/15], Step [1900/2100], Loss: 0.2225
Epoch [1/15], Step [2000/2100], Loss: 0.1269
Epoch [1/15], Step [2100/2100], Loss: 0.1783
Accuracy of the network on the 10000 test images: 91.5 %
Epoch [

Epoch [9/15], Step [500/2100], Loss: 0.0028
Epoch [9/15], Step [600/2100], Loss: 0.0713
Epoch [9/15], Step [700/2100], Loss: 0.0133
Epoch [9/15], Step [800/2100], Loss: 0.0373
Epoch [9/15], Step [900/2100], Loss: 0.0076
Epoch [9/15], Step [1000/2100], Loss: 0.0128
Epoch [9/15], Step [1100/2100], Loss: 0.0000
Epoch [9/15], Step [1200/2100], Loss: 0.3567
Epoch [9/15], Step [1300/2100], Loss: 0.0041
Epoch [9/15], Step [1400/2100], Loss: 0.0010
Epoch [9/15], Step [1500/2100], Loss: 0.0082
Epoch [9/15], Step [1600/2100], Loss: 0.0014
Epoch [9/15], Step [1700/2100], Loss: 0.0027
Epoch [9/15], Step [1800/2100], Loss: 0.2691
Epoch [9/15], Step [1900/2100], Loss: 0.1979
Epoch [9/15], Step [2000/2100], Loss: 0.0080
Epoch [9/15], Step [2100/2100], Loss: 0.0083
Accuracy of the network on the 10000 test images: 96.16666666666667 %
Epoch [10/15], Step [100/2100], Loss: 0.0162
Epoch [10/15], Step [200/2100], Loss: 0.0068
Epoch [10/15], Step [300/2100], Loss: 0.2289
Epoch [10/15], Step [400/2100], Los